In [161]:
from cirq_sic import *
from cirq.contrib.svg import SVGCircuit

# https://github.com/pifparfait/Efficient-Quantum-Modular-Arithmetics/blob/main/quantum_modular_artihmetics.ipynb

In [390]:
def F(q):
    yield cirq.qft(*q, without_reverse=False)

def Fdag(q):
    yield cirq.qft(*q, inverse=True, without_reverse=False)

def add_k(q, k):
    """
    Apply a rotation gate to each qubit in q rotating by an angle 'k * pi / 2^j'.
    """
    n = len(q)
    for j in range(n):
        #angle = k * np.pi  / (2**(n - 1 - j))
        angle = k * np.pi  / (2**j)
        yield cirq.Rz(rads=angle)(q[j])

def controlled_add_k(q, c, k):
    n = len(q)
    for j in range(n):
        #angle = k * np.pi  / (2**(n - 1 - j))
        angle = k * np.pi  / (2**j)
        yield cirq.Rz(rads=angle).on(q[j]).controlled_by(c)

def subtract_k(q, k):
    yield cirq.inverse(list(add_k(q, k)))

def add_k_mod_d(q, c, k, d, is_in_standard_basis=True):
    """
    Perform in-place modular addition of 'k' modulo d to the qubits in q. c is an auxiliary qubit.
    'is_in_standard_basis' specifies whether the input state is in the standard basis (True) or Fourier basis (False).
    """
    #sig_bit = q[-1]
    sig_bit = q[0]

    # Set the Fourier basis if 'is_in_standard_basis' is True
    if is_in_standard_basis:
        yield F(q)
    
    # Step 1: Add 'k' to the register
    yield add_k(q, k)
    
    # Step 2: Subtract 'd'
    yield add_k(q, -d)
    
    # Step 3: Conditionally add 'd' back
    yield Fdag(q)
    yield cirq.CNOT(sig_bit, c)
    yield F(q)

    # Step 4: Add 'd' to the register conditionally
    yield controlled_add_k(q, c, d)
    
    # Step 5: Clear the auxiliary bit
    yield add_k(q, -k)
    
    # Step 6: Conditionally add 'd' back
    yield Fdag(q)
    yield cirq.X(sig_bit)
    yield cirq.CNOT(sig_bit, c)  # Use the most significant bit to control the auxiliary bit
    yield cirq.X(sig_bit)
    yield F(q) 

    # Step 7: Add 'k' to the register
    yield add_k(q, k)
    
    # Recover the standard basis if 'is_in_standard_basis' is True
    if is_in_standard_basis:
        yield Fdag(q)

n = 3
d = 3
q = cirq.LineQubit.range(n)
c = cirq.NamedQubit("cntrl")
circ = cirq.Circuit((add_k_mod_d(q, c, 2, d)))
sim = cirq.Simulator()
ket = rand_ket(d)
initial_state = np.kron(np.concatenate([ket, np.zeros(2**n-d)]), np.eye(2)[0])
result = sim.simulate(circ, initial_state=initial_state, qubit_order=q+[c])
result.density_matrix_of(q)[:d, :d], np.diag(result.density_matrix_of([c]))

(array([[ 0.13 +0.j   , -0.148-0.095j, -0.241-0.155j],
        [-0.148+0.095j,  0.238+0.j   ,  0.388+0.002j],
        [-0.241+0.155j,  0.388-0.002j,  0.632+0.j   ]], dtype=complex64),
 array([1.+0.j, 0.+0.j], dtype=complex64))

This modular addition algorithm works by temporarily calculating values like a+k. Since a and k can both be as large as d-1, their sum can reach up to (d-1) + (d-1) = 2d-2.

Your qubit register can only represent integers from 0 to $2^n-1$. For the algorithm's logic (especially the part that checks for overflow using the most significant bit) to be reliable, the temporary sum must not exceed the register's capacity.

This leads to the requirement: $2d-2 < 2^n$. Thus we can encode a qudit in $n$ qubits with $d$ satisfying

$$d \leq 2^{n-1}  $$

In [363]:
d, 2**(n-1)

(4, 4)